# Order List Example Using Pandas Joins

This is an extended example based on the example originally in notebook 3.1.  

Here we are modeling a simple order system.  A store stocks SKUs (stock keeping units -- i.e., "things" that people buy).  Customers place orders and orders contain items.  In a system like this, the data are stored in separate tables that are combined for analysis and reporting.

In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

## Manually Create the DataFrames
See the bottom of the notebook for code that reads the DataFrames from csv files.

In [ ]:
# Customer list.  Note that this data would normally be stored in a data file (e.g., a csv file)
# or database table.  For simplicty, we'll build it here instead:
customers = pd.DataFrame({
     'customer_id' : [  'C01',     'C02',        'C03',    'C04',        'C05',      'C06',     'C07',    'C08']
    ,  'last_name' : [ 'Smith',  'Jones',    'Johnson',  'Black',      'Green',    'Kholm',  'Tester',  'Krock']
    , 'first_name' : [  'Jeff',   'Kale',       'Cara',  'Linda',     'Howard',    'Genie',    'John', 'Barbra']
    ,       'city' : ['Auburn', 'Mobile', 'Birmingham', 'Dothan', 'Greenville', 'Norcross', 'Columbus','Selina']
    ,      'state' : [    'AL',     'AL',         'AL',     'AL',         'AL',       'GA',       'OH',    'KS']
})
customers

In [ ]:
# SKU List - the item (SKU) information.
skus = pd.DataFrame({
      'sku' : [   'A100',    'A109',    'A200',    'A227',    'A300',    'A876',    'A904',   'A1021',   'A1776'],
    'descr' : ['Widget1', 'Widget2', 'Widget3', 'Widget4', 'Widget5', 'Widget6', 'Widget7', 'Widget8', 'Widget9'],
     'cost' : [    12.50,    423.50,      6.50,      6.34,   1850.45,      3.23,      7.50,     18.55,    127.22]})
skus

In [ ]:
# Order List
orders = pd.DataFrame({
           'order' : [       123,        456,        789,        823,        950,       1024,       1223,       1552]
    ,'customer_id' : [     'C01',      'C05',      'C04',      'C01',      'C03',      'C03',      'C06',      'C02']
    , 'order_date' : ['01/07/17', '01/09/17', '01/09/17', '01/10/17', '01/15/17', '01/16/17', '01/16/17', '01/17/17']
})
orders

In [ ]:
# And we have a separate dataframe with the items in each order
items = pd.DataFrame({
        'order' : [   123,    123,    123,    456,    456,    789,    823,     950,    950,   1024, 1223, 1223, 1552],
          'sku' : ['A109', 'A100', 'A200', 'A109', 'A227', 'A109', 'A100',  'A300', 'A904', 'A200', 'A300', 'A1021', 'A876'],
        'price' : [765.55, 227.83,  12.50, 665.55,  10.68, 760.00, 225.55, 2650.55,  15.22,  12.25, 10.25, 17.50, 42.75]})
items

In [ ]:
# First, join (merge) the customers with the orders
pd.merge(customers,orders)
# Why not store all of this information in a single file?

In [ ]:
# We'd like to join (merge) the data so that the item information can be 
# easily combined with the order information.
pd.merge(pd.merge(customers,orders), items)

In [ ]:
# And now add in the item/SKU information (and save the resulting dataframe)
sales = pd.merge(pd.merge(pd.merge(customers,orders), items),skus)
sales

In [ ]:
# now that we have the data we need, let's add a calculated column
sales['profit'] = sales['price'] - sales['cost']
sales

In [ ]:
# Grab all my stuff ...
sales[sales.first_name == 'Jeff']

In [ ]:
# Total the profit ... How does this work (i.e., can you explain steps?
# What an awesome test question this would be ... :-)).
sales[sales.first_name == 'Jeff'].profit.sum()

In [ ]:
# All items sold with profit > 250
sales[sales.profit > 250]

In [ ]:
# What if we use a left join on the second merge/join ...
a = pd.merge(skus, pd.merge(pd.merge(customers,orders), items), how="left")
a

In [ ]:
# Now, suppose that we define a mask to identify those entries
# with no orders ...
pd.isna(a['order'])

In [ ]:
# ... and then apply that mask to the left-join dataset ...
a[pd.isna(a['customer_id'])][['sku', 'descr', 'cost']]
# to produce a list of SKUs that haven't been ordered

In [ ]:
# What about customers with no orders?
b = pd.merge(customers,orders, how="left")
b[pd.isna(b['order'])]

In [ ]:
# orders on 01/17/17
sales[sales.order_date == '01/17/17']

In [ ]:
# orders for SKU A227
sales[sales.sku == 'A227']

## Reading the Data Frames from csv Files


In [ ]:
customers = pd.read_csv("../data/orders/customers.csv")
customers

In [ ]:
orders = pd.read_csv("../data/orders/orders.csv")
orders

In [ ]:
skus = pd.read_csv("../data/orders/skus.csv")
skus

In [ ]:
items = pd.read_csv("../data/orders/items.csv")
items

In [ ]:
# re-create the sales data frame
sales = pd.merge(pd.merge(pd.merge(customers,orders), items),skus)
sales['profit'] = sales['price'] - sales['cost']
sales

## Aggregation and Grouping

Now that we have "complete" dataframe, we can use the aggregation and grouping functions to analyze the data.

Reference the "split, apply, combine" approace from the VP book and the couse slide set

In [ ]:
# Cost and profit by Customer
sales.groupby('customer_id')[['cost','profit']].sum()

In [ ]:
# Profit by SKU
sales.groupby('sku')[['profit']].sum()

In [ ]:
# Profit by State
sales.groupby('state')[['profit']].sum()

In [ ]:
# By customer_id and sku
sales.groupby(['customer_id', 'sku'])[['profit']].sum()

In [ ]:
# More aggregate vales by SKU
sales.groupby('sku')[['profit']].aggregate(['sum', 'mean', 'median'])